# DASK 2026 - Twin Towers Model

İkiz kule modeli - DASK Depreme Dayanıklı Bina Tasarımı Yarışması 2026

## Specifications
- **Twin Towers**: 80mm gap between towers
- **Bridges**: 3-4 bridges connecting towers
- **Top Bridge**: 2-story bridge at top two floors (mandatory)
- **Max floors**: 30 (including ground floor)
- **Floor height**: 60mm (ground floor: 90mm)

---
## 1. Import Libraries and Setup

In [98]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

---
## 2. Building Geometry Parameters

### DASK 2026 Maket Scale
- Floor height: 60mm (ground: 90mm)
- Max plan: 160mm × 400mm per tower
- Gap between towers: 80mm
- Min plan: 80mm × 200mm

### Model Scale (1:100)
- Floor height: 6m (ground: 9m)
- Bay width: 8m = 80mm in maket

In [99]:
# ============================================
# GEOMETRY PARAMETERS (Model Scale - meters)
# ============================================
BAY_WIDTH = 8.0  # 8m = 80mm in maket

# Tower dimensions
PODIUM_BAYS_X = 5    # 5 bays × 8m = 40m = 400mm (max allowed)
PODIUM_BAYS_Y = 2    # 2 bays × 8m = 16m = 160mm (max allowed)
PODIUM_FLOORS = 13   # Floors 0-12

TOWER_BAYS_X = 3     # 3 bays × 8m = 24m = 240mm
TOWER_BAYS_Y = 2     # 2 bays × 8m = 16m = 160mm
TOTAL_FLOORS = 26    # Floors 0-25

GROUND_FLOOR_HEIGHT = 9.0   # 9m = 90mm in maket
TYPICAL_FLOOR_HEIGHT = 6.0  # 6m = 60mm in maket

# Twin tower gap
TOWER_GAP = 8.0  # 8m = 80mm in maket (between tower edges)

# Tower 1 position (left tower)
TOWER1_ORIGIN_X = 0.0
TOWER1_ORIGIN_Y = 0.0

# Tower 2 position (right tower) - 80mm gap in Y direction
# Tower 1 Y extent: 0 to 16m
# Gap: 8m (80mm)
# Tower 2 Y start: 16 + 8 = 24m
TOWER2_ORIGIN_X = 0.0
TOWER2_ORIGIN_Y = PODIUM_BAYS_Y * BAY_WIDTH + TOWER_GAP  # 16 + 8 = 24m

# Coordinate arrays for each tower
PODIUM_X_COORDS = np.arange(PODIUM_BAYS_X + 1) * BAY_WIDTH
TOWER_X_COORDS = np.array([8.0, 16.0, 24.0, 32.0])

# Y coords for Tower 1
Y_COORDS_T1 = np.arange(PODIUM_BAYS_Y + 1) * BAY_WIDTH + TOWER1_ORIGIN_Y

# Y coords for Tower 2
Y_COORDS_T2 = np.arange(PODIUM_BAYS_Y + 1) * BAY_WIDTH + TOWER2_ORIGIN_Y

# Z coordinates (same for both towers)
Z_COORDS = np.zeros(TOTAL_FLOORS)
Z_COORDS[0] = 0.0
Z_COORDS[1] = GROUND_FLOOR_HEIGHT
Z_COORDS[2:] = GROUND_FLOOR_HEIGHT + np.arange(1, TOTAL_FLOORS - 1) * TYPICAL_FLOOR_HEIGHT

print("TOWER 1 (Left):")
print(f"  X: {PODIUM_X_COORDS[0]:.0f} to {PODIUM_X_COORDS[-1]:.0f} m")
print(f"  Y: {Y_COORDS_T1[0]:.0f} to {Y_COORDS_T1[-1]:.0f} m")

print(f"\nTOWER 2 (Right):")
print(f"  X: {PODIUM_X_COORDS[0]:.0f} to {PODIUM_X_COORDS[-1]:.0f} m")
print(f"  Y: {Y_COORDS_T2[0]:.0f} to {Y_COORDS_T2[-1]:.0f} m")

print(f"\nGap between towers: {TOWER_GAP:.0f} m = {TOWER_GAP*10:.0f} mm in maket")
print(f"Total Y extent: {Y_COORDS_T2[-1]:.0f} m = {Y_COORDS_T2[-1]*10:.0f} mm in maket")
print(f"\nZ_COORDS: {Z_COORDS}")

TOWER 1 (Left):
  X: 0 to 40 m
  Y: 0 to 16 m

TOWER 2 (Right):
  X: 0 to 40 m
  Y: 24 to 40 m

Gap between towers: 8 m = 80 mm in maket
Total Y extent: 40 m = 400 mm in maket

Z_COORDS: [  0.   9.  15.  21.  27.  33.  39.  45.  51.  57.  63.  69.  75.  81.
  87.  93.  99. 105. 111. 117. 123. 129. 135. 141. 147. 153.]


---
## 3. Generate Position Matrix for Twin Towers

In [137]:
def generate_tower_nodes(tower_id, y_coords, x_podium, x_tower, z_coords, 
                         podium_floors, total_floors, start_node_id=0):
    """
    Generate nodes for a single tower.
    Returns: node data list, node_lookup dict, next_node_id
    """
    nodes = []
    node_lookup = {}
    node_id = start_node_id
    
    # Podium nodes: floors 0 to podium_floors-1
    for f in range(podium_floors):
        for x in x_podium:
            for y in y_coords:
                nodes.append({
                    'node_id': node_id,
                    'x': x, 'y': y, 'z': z_coords[f],
                    'floor': f,
                    'zone': 'podium',
                    'tower': tower_id
                })
                node_lookup[(tower_id, f, x, y)] = node_id
                node_id += 1
    
    # Tower nodes: floors podium_floors to total_floors-1
    for f in range(podium_floors, total_floors):
        for x in x_tower:
            for y in y_coords:
                nodes.append({
                    'node_id': node_id,
                    'x': x, 'y': y, 'z': z_coords[f],
                    'floor': f,
                    'zone': 'tower',
                    'tower': tower_id
                })
                node_lookup[(tower_id, f, x, y)] = node_id
                node_id += 1
    
    # Chevron center nodes removed - using full 2-floor X-braces now
    
    return nodes, node_lookup, node_id

# Generate nodes for Tower 1
nodes_t1, lookup_t1, next_id = generate_tower_nodes(
    tower_id=1,
    y_coords=Y_COORDS_T1,
    x_podium=PODIUM_X_COORDS,
    x_tower=TOWER_X_COORDS,
    z_coords=Z_COORDS,
    podium_floors=PODIUM_FLOORS,
    total_floors=TOTAL_FLOORS,
    start_node_id=0
)
print(f"Tower 1: {len(nodes_t1)} nodes (IDs: 0 to {next_id-1})")

# Generate nodes for Tower 2
nodes_t2, lookup_t2, next_id = generate_tower_nodes(
    tower_id=2,
    y_coords=Y_COORDS_T2,
    x_podium=PODIUM_X_COORDS,
    x_tower=TOWER_X_COORDS,
    z_coords=Z_COORDS,
    podium_floors=PODIUM_FLOORS,
    total_floors=TOTAL_FLOORS,
    start_node_id=next_id
)
print(f"Tower 2: {len(nodes_t2)} nodes (IDs: {next_id - len(nodes_t2)} to {next_id-1})")

# Combine all nodes
all_nodes = nodes_t1 + nodes_t2
position_df = pd.DataFrame(all_nodes)

# Combined lookup
node_lookup = {**lookup_t1, **lookup_t2}

# Create coords array
n_nodes = len(position_df)
coords = position_df[['x', 'y', 'z']].values

print(f"\nTotal nodes: {n_nodes}")
print(f"Position Matrix Shape: {position_df.shape}")
display(position_df.head(10))

Tower 1: 390 nodes (IDs: 0 to 389)
Tower 2: 390 nodes (IDs: 390 to 779)

Total nodes: 780
Position Matrix Shape: (780, 7)


,node_id,x,y,z,floor,zone,tower
0,0,0.0,0.0,0.0,0,podium,1
1,1,0.0,8.0,0.0,0,podium,1
2,2,0.0,16.0,0.0,0,podium,1
3,3,8.0,0.0,0.0,0,podium,1
4,4,8.0,8.0,0.0,0,podium,1
5,5,8.0,16.0,0.0,0,podium,1
6,6,16.0,0.0,0.0,0,podium,1
7,7,16.0,8.0,0.0,0,podium,1
8,8,16.0,16.0,0.0,0,podium,1
9,9,24.0,0.0,0.0,0,podium,1


---
## 4. Generate Connectivity Matrix for Twin Towers

In [ ]:
def generate_tower_elements(tower_id, y_coords, x_podium, x_tower, z_coords,
                            podium_floors, total_floors, node_lookup, start_elem_id=0):
    """
    Generate elements for a single tower.
    Returns: elements list, next_elem_id
    
    IMPROVED BRACING:
    - Podium: DENSE bracing at every column space
    - Tower: Proper DAMA (checkerboard) pattern
    - Core: X-braces at critical floors
    """
    elements = []
    elem_id = start_elem_id
    
    def add_element(n1, n2, etype):
        nonlocal elem_id
        if n1 is not None and n2 is not None:
            elements.append({
                'element_id': elem_id,
                'node_i': n1,
                'node_j': n2,
                'element_type': etype,
                'tower': tower_id
            })
            elem_id += 1
            return True
        return False
    
    def get_node(f, x, y):
        return node_lookup.get((tower_id, f, x, y))
    
    # ============================================
    # BEAMS X-DIRECTION
    # ============================================
    # Podium beams X
    for f in range(podium_floors):
        for y in y_coords:
            for i in range(len(x_podium) - 1):
                add_element(get_node(f, x_podium[i], y), 
                           get_node(f, x_podium[i+1], y), 'beam_x')
    
    # Tower beams X
    for f in range(podium_floors, total_floors):
        for y in y_coords:
            for i in range(len(x_tower) - 1):
                add_element(get_node(f, x_tower[i], y),
                           get_node(f, x_tower[i+1], y), 'beam_x')
    
    # ============================================
    # BEAMS Y-DIRECTION
    # ============================================
    # Podium beams Y
    for f in range(podium_floors):
        for x in x_podium:
            for i in range(len(y_coords) - 1):
                add_element(get_node(f, x, y_coords[i]),
                           get_node(f, x, y_coords[i+1]), 'beam_y')
    
    # Tower beams Y
    for f in range(podium_floors, total_floors):
        for x in x_tower:
            for i in range(len(y_coords) - 1):
                add_element(get_node(f, x, y_coords[i]),
                           get_node(f, x, y_coords[i+1]), 'beam_y')
    
    # ============================================
    # COLUMNS
    # ============================================
    # Podium columns
    for f in range(podium_floors - 1):
        for x in x_podium:
            for y in y_coords:
                add_element(get_node(f, x, y), get_node(f+1, x, y), 'column')
    
    # Transition columns (podium to tower)
    for x in x_tower:
        for y in y_coords:
            add_element(get_node(podium_floors-1, x, y),
                       get_node(podium_floors, x, y), 'column')
    
    # Tower columns
    for f in range(podium_floors, total_floors - 1):
        for x in x_tower:
            for y in y_coords:
                add_element(get_node(f, x, y), get_node(f+1, x, y), 'column')
    
    # ============================================
    # PODIUM BRACES XZ - DENSE (Every column space!)
    # Front/Back faces, 2-story span, ALL bays
    # ============================================
    for f in range(0, podium_floors - 1, 2):  # 0, 2, 4, 6, 8, 10
        f_top = f + 2
        if f_top >= podium_floors:
            f_top = podium_floors - 1
            if f_top <= f:
                continue
        
        for y in y_coords:  # Front and back faces
            # ALL podium bays - dense bracing
            for i in range(len(x_podium) - 1):
                x1, x2 = x_podium[i], x_podium[i+1]
                
                n_bl = get_node(f, x1, y)
                n_br = get_node(f, x2, y)
                n_tl = get_node(f_top, x1, y)
                n_tr = get_node(f_top, x2, y)
                
                if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                    add_element(n_bl, n_tr, 'brace_xz')
                    add_element(n_br, n_tl, 'brace_xz')
    
    # ============================================
    # TOWER BRACES XZ - DAMA (Checkerboard pattern)
    # Front/Back faces, 2-story span
    # Alternating pattern: (floor_group + bay_index) % 2 == 0
    # ============================================
    for f in range(podium_floors - 1, total_floors - 1, 2):  # Start from transition
        f_top = f + 2
        if f_top >= total_floors:
            break
        
        floor_group = (f - podium_floors + 1) // 2  # 0, 1, 2, 3...
        
        for y_idx, y in enumerate(y_coords):  # Front and back faces
            for i in range(len(x_tower) - 1):
                x1, x2 = x_tower[i], x_tower[i+1]
                
                n_bl = get_node(f, x1, y)
                n_br = get_node(f, x2, y)
                n_tl = get_node(f_top, x1, y)
                n_tr = get_node(f_top, x2, y)
                
                if not all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                    continue
                
                # TRUE DAMA PATTERN: alternating checkerboard
                # Different pattern for front (y_idx=0) and back (y_idx=2) faces
                if (floor_group + i + y_idx) % 2 == 0:
                    add_element(n_bl, n_tr, 'brace_xz')
                    add_element(n_br, n_tl, 'brace_xz')
    
    # ============================================
    # PODIUM BRACES YZ - DENSE (Side faces)
    # Left/Right edges, every 2 floors
    # ============================================
    for f in range(0, podium_floors - 1, 2):
        f_top = f + 2
        if f_top >= podium_floors:
            f_top = podium_floors - 1
            if f_top <= f:
                continue
        
        # Podium side edges: X=0 and X=40
        for x in [x_podium[0], x_podium[-1]]:
            for j in range(len(y_coords) - 1):
                y1, y2 = y_coords[j], y_coords[j+1]
                
                n_bl = get_node(f, x, y1)
                n_br = get_node(f, x, y2)
                n_tl = get_node(f_top, x, y1)
                n_tr = get_node(f_top, x, y2)
                
                if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                    add_element(n_bl, n_tr, 'brace_yz')
                    add_element(n_br, n_tl, 'brace_yz')
    
    # ============================================
    # TOWER BRACES YZ - DAMA (Side faces)
    # Left/Right edges X=8, X=32
    # ============================================
    for f in range(podium_floors - 1, total_floors - 1, 2):
        f_top = f + 2
        if f_top >= total_floors:
            break
        
        floor_group = (f - podium_floors + 1) // 2
        
        # Tower side edges: X=8 and X=32
        for face_idx, x in enumerate([x_tower[0], x_tower[-1]]):
            for j in range(len(y_coords) - 1):
                y1, y2 = y_coords[j], y_coords[j+1]
                
                # Dama pattern for YZ braces too
                if (floor_group + face_idx + j) % 2 != 0:
                    continue
                
                n_bl = get_node(f, x, y1)
                n_br = get_node(f, x, y2)
                n_tl = get_node(f_top, x, y1)
                n_tr = get_node(f_top, x, y2)
                
                if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                    add_element(n_bl, n_tr, 'brace_yz')
                    add_element(n_br, n_tl, 'brace_yz')
    
    # ============================================
    # CORE WALLS (YZ planes at X=16 and X=24)
    # At bridge floors and key locations
    # ============================================
    core_x_left = 16.0
    core_x_right = 24.0
    core_y_front = y_coords[0]
    core_y_back = y_coords[-1]
    core_y_mid = y_coords[1] if len(y_coords) > 2 else (y_coords[0] + y_coords[-1]) / 2
    
    # Core braces at key floors
    CORE_WALL_FLOORS = [0, 5, 11, 17, 23, 24]  # Ground + bridge floors + top
    for f in CORE_WALL_FLOORS:
        if f >= total_floors - 1:
            continue
        f_top = f + 1
        
        # Left core wall (X=16)
        for y1, y2 in [(core_y_front, core_y_mid), (core_y_mid, core_y_back)]:
            n_bl = get_node(f, core_x_left, y1)
            n_br = get_node(f, core_x_left, y2)
            n_tl = get_node(f_top, core_x_left, y1)
            n_tr = get_node(f_top, core_x_left, y2)
            if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                add_element(n_bl, n_tr, 'core_wall')
                add_element(n_br, n_tl, 'core_wall')
        
        # Right core wall (X=24)
        for y1, y2 in [(core_y_front, core_y_mid), (core_y_mid, core_y_back)]:
            n_bl = get_node(f, core_x_right, y1)
            n_br = get_node(f, core_x_right, y2)
            n_tl = get_node(f_top, core_x_right, y1)
            n_tr = get_node(f_top, core_x_right, y2)
            if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                add_element(n_bl, n_tr, 'core_wall')
                add_element(n_br, n_tl, 'core_wall')
    
    # ============================================
    # FLOOR BRACES (Diaphragm) - Selected floors only
    # ============================================
    DIAPHRAGM_FLOORS = [0, 6, 12, 18, 24]  # Ground + bridge connection floors
    
    for f in DIAPHRAGM_FLOORS:
        if f >= total_floors:
            continue
        
        # Podium floor braces
        if f < podium_floors:
            for i in range(len(x_podium) - 1):
                for j in range(len(y_coords) - 1):
                    # Dama pattern for floor braces
                    if (i + j) % 2 != 0:
                        continue
                    
                    x1, x2 = x_podium[i], x_podium[i+1]
                    y1, y2 = y_coords[j], y_coords[j+1]
                    
                    n_bl = get_node(f, x1, y1)
                    n_br = get_node(f, x2, y1)
                    n_tl = get_node(f, x1, y2)
                    n_tr = get_node(f, x2, y2)
                    
                    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                        add_element(n_bl, n_tr, 'brace_floor')
                        add_element(n_br, n_tl, 'brace_floor')
        else:
            # Tower floor braces
            for i in range(len(x_tower) - 1):
                for j in range(len(y_coords) - 1):
                    if (i + j) % 2 != 0:
                        continue
                    
                    x1, x2 = x_tower[i], x_tower[i+1]
                    y1, y2 = y_coords[j], y_coords[j+1]
                    
                    n_bl = get_node(f, x1, y1)
                    n_br = get_node(f, x2, y1)
                    n_tl = get_node(f, x1, y2)
                    n_tr = get_node(f, x2, y2)
                    
                    if all(n is not None for n in [n_bl, n_br, n_tl, n_tr]):
                        add_element(n_bl, n_tr, 'brace_floor')
                        add_element(n_br, n_tl, 'brace_floor')
    
    # ============================================
    # SPACE BRACES (Podium to Tower transition)
    # ============================================
    f_pod_top = podium_floors - 1
    f_tow_bot = podium_floors
    
    transitions = [(0.0, 8.0), (40.0, 32.0)]
    for x_pod, x_tow in transitions:
        for y in y_coords:
            n_pod = get_node(f_pod_top, x_pod, y)
            for y_tow in y_coords:
                n_tow = get_node(f_tow_bot, x_tow, y_tow)
                add_element(n_pod, n_tow, 'brace_space')
    
    return elements, elem_id

# Generate elements for Tower 1
elements_t1, next_elem_id = generate_tower_elements(
    tower_id=1,
    y_coords=Y_COORDS_T1,
    x_podium=PODIUM_X_COORDS,
    x_tower=TOWER_X_COORDS,
    z_coords=Z_COORDS,
    podium_floors=PODIUM_FLOORS,
    total_floors=TOTAL_FLOORS,
    node_lookup=node_lookup,
    start_elem_id=0
)
print(f"Tower 1: {len(elements_t1)} elements")

# Generate elements for Tower 2
elements_t2, next_elem_id = generate_tower_elements(
    tower_id=2,
    y_coords=Y_COORDS_T2,
    x_podium=PODIUM_X_COORDS,
    x_tower=TOWER_X_COORDS,
    z_coords=Z_COORDS,
    podium_floors=PODIUM_FLOORS,
    total_floors=TOTAL_FLOORS,
    node_lookup=node_lookup,
    start_elem_id=next_elem_id
)
print(f"Tower 2: {len(elements_t2)} elements")

---
## 5. Generate Bridge Connections

DASK 2026 Requirements:
- 3-4 bridges between towers (1/4, 1/2, 3/4 height + top)
- Top bridge: **2-story** at top two floors (mandatory)
- Other bridges: single-story at 1/4, 1/2, 3/4 building height
- Bridges must not get support from other floors

### Bridge Design:
- **Top/Bottom faces**: X-braces (horizontal plane bracing)
- **Side faces**: Warren truss pattern
- **Connection type**: PIN (moment-free, rotation allowed)

In [139]:
# ============================================
# BRIDGE CONNECTIONS BETWEEN TOWERS
# ============================================
# DASK 2026: Köprüler alt-üst kirişleri ve çapraz bağları olan gerçek kafes kirişler olmalı
# Single-story: 1 kat span (floor N-1 → floor N arası kutu)
# 2-story: 2 kat span (floor 24 → floor 25 arası kutu)

# Building height calculation
BUILDING_HEIGHT = Z_COORDS[-1]  # Top floor Z
print(f"Building height: {BUILDING_HEIGHT:.0f}m = {BUILDING_HEIGHT*10:.0f}mm in maket")

# Bridge floors - (bottom_floor, top_floor) çiftleri olarak tanımla
# Single-story bridges: 1 kat span at 1/4, 1/2, 3/4 heights
BRIDGE_FLOORS_SINGLE = [(5, 6), (11, 12), (17, 18)]   # 1/4, 1/2, 3/4 height (1-story span each)

# 2-story bridge: 2 kat span at top (Floor 23 → Floor 25 = 2 floor heights)
BRIDGE_FLOORS_DOUBLE = (23, 25)  # Top 2-story bridge (mandatory) - spans 2 floors vertically

# Bridge X positions (width of bridge in X direction)
BRIDGE_X_LEFT = 16.0    # Core left edge
BRIDGE_X_RIGHT = 24.0   # Core right edge
BRIDGE_WIDTH = BRIDGE_X_RIGHT - BRIDGE_X_LEFT  # 8m = 80mm

# Y positions at tower edges
Y_T1_BACK = Y_COORDS_T1[-1]    # Tower 1 back edge (Y=16)
Y_T2_FRONT = Y_COORDS_T2[0]    # Tower 2 front edge (Y=24)
BRIDGE_LENGTH = Y_T2_FRONT - Y_T1_BACK  # 8m = 80mm

print(f"\nBridge Configuration:")
print(f"  Single-story bridges (1-floor span): {BRIDGE_FLOORS_SINGLE}")
print(f"  2-story bridge: {BRIDGE_FLOORS_DOUBLE}")
print(f"  Bridge X: {BRIDGE_X_LEFT:.0f} to {BRIDGE_X_RIGHT:.0f}m (width: {BRIDGE_WIDTH:.0f}m)")
print(f"  Bridge Y: {Y_T1_BACK:.0f} to {Y_T2_FRONT:.0f}m (span: {BRIDGE_LENGTH:.0f}m)")
print(f"  Height per floor: {TYPICAL_FLOOR_HEIGHT:.0f}m")

# Helper functions
def get_t1_node(f, x, y):
    return node_lookup.get((1, f, x, y))

def get_t2_node(f, x, y):
    return node_lookup.get((2, f, x, y))

# Initialize bridge elements and nodes
bridge_elements = []
bridge_nodes = []
bridge_node_lookup = {}
bridge_node_id = n_nodes  # Continue from last tower node ID

elem_id = next_elem_id

# ============================================
# CREATE BRIDGE NODES (Mid-span nodes for trusses)
# ============================================
print(f"\nCreating bridge nodes...")

def add_bridge_node(floor_idx, x, y):
    """Add a new bridge node at given position"""
    global bridge_node_id
    z = Z_COORDS[floor_idx]
    
    key = ('bridge', floor_idx, x, y)
    if key not in bridge_node_lookup:
        bridge_nodes.append({
            'node_id': bridge_node_id,
            'x': x, 'y': y, 'z': z,
            'floor': floor_idx,
            'zone': 'bridge'
        })
        bridge_node_lookup[key] = bridge_node_id
        bridge_node_id += 1
    
    return bridge_node_lookup[key]

def get_bridge_node(floor_idx, x, y):
    """Get bridge node if exists"""
    return bridge_node_lookup.get(('bridge', floor_idx, x, y))

# Create mid-span nodes for all bridge floors (at Y = mid-point of bridge)
Y_BRIDGE_MID = (Y_T1_BACK + Y_T2_FRONT) / 2  # Y = 20m

# Single-story bridges: need nodes at both bottom and top floors
for floor_bot, floor_top in BRIDGE_FLOORS_SINGLE:
    add_bridge_node(floor_bot, BRIDGE_X_LEFT, Y_BRIDGE_MID)
    add_bridge_node(floor_bot, BRIDGE_X_RIGHT, Y_BRIDGE_MID)
    add_bridge_node(floor_top, BRIDGE_X_LEFT, Y_BRIDGE_MID)
    add_bridge_node(floor_top, BRIDGE_X_RIGHT, Y_BRIDGE_MID)

# 2-story bridge: nodes at both floors
floor_bot_2s, floor_top_2s = BRIDGE_FLOORS_DOUBLE
add_bridge_node(floor_bot_2s, BRIDGE_X_LEFT, Y_BRIDGE_MID)
add_bridge_node(floor_bot_2s, BRIDGE_X_RIGHT, Y_BRIDGE_MID)
add_bridge_node(floor_top_2s, BRIDGE_X_LEFT, Y_BRIDGE_MID)
add_bridge_node(floor_top_2s, BRIDGE_X_RIGHT, Y_BRIDGE_MID)

print(f"  Created {len(bridge_nodes)} bridge nodes")

# ============================================
# BOX TRUSS BRIDGE GENERATOR (universal for 1 or 2 story span)
# ============================================
def create_box_bridge(floor_bot, floor_top, bridge_name="Bridge"):
    """
    Create a BOX TRUSS bridge with:
    - Complete beam grid at BOTH floor levels (bottom and top)
    - Vertical columns at ALL 6 positions (4 corners + 2 mid-span)
    - X-braces on top and bottom horizontal planes
    - Warren truss on both side faces (X constant)
    - X-braces on front and back faces (Y constant)
    - Pin connections at tower interfaces
    
    Works for any floor span (1-story, 2-story, etc.)
    """
    global elem_id
    elements = []
    
    # Get nodes for both floors
    nodes = {}
    for f, label in [(floor_bot, 'bot'), (floor_top, 'top')]:
        nodes[f'{label}_t1_left'] = get_t1_node(f, BRIDGE_X_LEFT, Y_T1_BACK)
        nodes[f'{label}_t1_right'] = get_t1_node(f, BRIDGE_X_RIGHT, Y_T1_BACK)
        nodes[f'{label}_t2_left'] = get_t2_node(f, BRIDGE_X_LEFT, Y_T2_FRONT)
        nodes[f'{label}_t2_right'] = get_t2_node(f, BRIDGE_X_RIGHT, Y_T2_FRONT)
        nodes[f'{label}_mid_left'] = get_bridge_node(f, BRIDGE_X_LEFT, Y_BRIDGE_MID)
        nodes[f'{label}_mid_right'] = get_bridge_node(f, BRIDGE_X_RIGHT, Y_BRIDGE_MID)
    
    missing = [k for k, v in nodes.items() if v is None]
    if missing:
        print(f"  Warning: {bridge_name} missing nodes: {missing}")
        return elements
    
    height = Z_COORDS[floor_top] - Z_COORDS[floor_bot]
    print(f"  {bridge_name} (Floor {floor_bot}→{floor_top}, height={height:.0f}m): ", end="")
    
    # ==========================================
    # BOTTOM FLOOR BEAMS
    # ==========================================
    # Edge beams along X at tower edges
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t1_left'], 'node_j': nodes['bot_t1_right'],
        'element_type': 'bridge_beam', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t2_left'], 'node_j': nodes['bot_t2_right'],
        'element_type': 'bridge_beam', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    
    # Mid-span beam along X
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_mid_left'], 'node_j': nodes['bot_mid_right'],
        'element_type': 'bridge_beam', 'tower': 'bridge', 'connection': 'rigid'
    })
    elem_id += 1
    
    # Spanning beams along Y (left and right sides)
    for side in ['left', 'right']:
        for seg in [('t1', 'mid'), ('mid', 't2')]:
            n1 = nodes[f'bot_{seg[0]}_{side}']
            n2 = nodes[f'bot_{seg[1]}_{side}']
            elements.append({
                'element_id': elem_id, 'node_i': n1, 'node_j': n2,
                'element_type': 'bridge_beam', 'tower': 'bridge', 'connection': 'pin'
            })
            elem_id += 1
    
    # ==========================================
    # TOP FLOOR BEAMS
    # ==========================================
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_t1_left'], 'node_j': nodes['top_t1_right'],
        'element_type': 'bridge_beam', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_t2_left'], 'node_j': nodes['top_t2_right'],
        'element_type': 'bridge_beam', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_mid_left'], 'node_j': nodes['top_mid_right'],
        'element_type': 'bridge_beam', 'tower': 'bridge', 'connection': 'rigid'
    })
    elem_id += 1
    
    for side in ['left', 'right']:
        for seg in [('t1', 'mid'), ('mid', 't2')]:
            n1 = nodes[f'top_{seg[0]}_{side}']
            n2 = nodes[f'top_{seg[1]}_{side}']
            elements.append({
                'element_id': elem_id, 'node_i': n1, 'node_j': n2,
                'element_type': 'bridge_beam', 'tower': 'bridge', 'connection': 'pin'
            })
            elem_id += 1
    
    # ==========================================
    # VERTICAL COLUMNS AT ALL 6 POSITIONS
    # ==========================================
    # 4 corner columns + 2 mid-span columns = 6 total
    vertical_pairs = [
        ('bot_t1_left', 'top_t1_left'),     # Tower 1, left corner
        ('bot_t1_right', 'top_t1_right'),   # Tower 1, right corner
        ('bot_t2_left', 'top_t2_left'),     # Tower 2, left corner
        ('bot_t2_right', 'top_t2_right'),   # Tower 2, right corner
        ('bot_mid_left', 'top_mid_left'),   # Mid-span, left
        ('bot_mid_right', 'top_mid_right'), # Mid-span, right
    ]
    
    for bot_key, top_key in vertical_pairs:
        elements.append({
            'element_id': elem_id, 
            'node_i': nodes[bot_key], 
            'node_j': nodes[top_key],
            'element_type': 'bridge_column', 
            'tower': 'bridge', 
            'connection': 'pin' if 't1' in bot_key or 't2' in bot_key else 'rigid'
        })
        elem_id += 1
    
    # ==========================================
    # BOTTOM PLANE X-BRACES (Horizontal @ bottom floor)
    # ==========================================
    # Bay 1: T1 to Mid
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t1_left'], 'node_j': nodes['bot_mid_right'],
        'element_type': 'bridge_brace_bot', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t1_right'], 'node_j': nodes['bot_mid_left'],
        'element_type': 'bridge_brace_bot', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    # Bay 2: Mid to T2
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_mid_left'], 'node_j': nodes['bot_t2_right'],
        'element_type': 'bridge_brace_bot', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_mid_right'], 'node_j': nodes['bot_t2_left'],
        'element_type': 'bridge_brace_bot', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    
    # ==========================================
    # TOP PLANE X-BRACES (Horizontal @ top floor)
    # ==========================================
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_t1_left'], 'node_j': nodes['top_mid_right'],
        'element_type': 'bridge_brace_top', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_t1_right'], 'node_j': nodes['top_mid_left'],
        'element_type': 'bridge_brace_top', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_mid_left'], 'node_j': nodes['top_t2_right'],
        'element_type': 'bridge_brace_top', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_mid_right'], 'node_j': nodes['top_t2_left'],
        'element_type': 'bridge_brace_top', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    
    # ==========================================
    # LEFT SIDE FACE - WARREN TRUSS (X = 16m, YZ plane)
    # ==========================================
    # T1 to Mid bay
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t1_left'], 'node_j': nodes['top_mid_left'],
        'element_type': 'bridge_truss_side', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_t1_left'], 'node_j': nodes['bot_mid_left'],
        'element_type': 'bridge_truss_side', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    # Mid to T2 bay
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_mid_left'], 'node_j': nodes['top_t2_left'],
        'element_type': 'bridge_truss_side', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_mid_left'], 'node_j': nodes['bot_t2_left'],
        'element_type': 'bridge_truss_side', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    
    # ==========================================
    # RIGHT SIDE FACE - WARREN TRUSS (X = 24m, YZ plane)
    # ==========================================
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t1_right'], 'node_j': nodes['top_mid_right'],
        'element_type': 'bridge_truss_side', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_t1_right'], 'node_j': nodes['bot_mid_right'],
        'element_type': 'bridge_truss_side', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_mid_right'], 'node_j': nodes['top_t2_right'],
        'element_type': 'bridge_truss_side', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['top_mid_right'], 'node_j': nodes['bot_t2_right'],
        'element_type': 'bridge_truss_side', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    
    # ==========================================
    # FRONT FACE X-BRACES (Y = T1_BACK = 16m, XZ plane)
    # ==========================================
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t1_left'], 'node_j': nodes['top_t1_right'],
        'element_type': 'bridge_brace_face', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t1_right'], 'node_j': nodes['top_t1_left'],
        'element_type': 'bridge_brace_face', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    
    # ==========================================
    # BACK FACE X-BRACES (Y = T2_FRONT = 24m, XZ plane)
    # ==========================================
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t2_left'], 'node_j': nodes['top_t2_right'],
        'element_type': 'bridge_brace_face', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    elements.append({
        'element_id': elem_id, 'node_i': nodes['bot_t2_right'], 'node_j': nodes['top_t2_left'],
        'element_type': 'bridge_brace_face', 'tower': 'bridge', 'connection': 'pin'
    })
    elem_id += 1
    
    print(f"{len(elements)} elements")
    return elements

# ============================================
# CREATE ALL BRIDGES
# ============================================
print("\nCreating bridges (all with box truss structure)...")

# Single-story bridges (1-floor span each) at 1/4, 1/2, 3/4 heights
for i, (floor_bot, floor_top) in enumerate(BRIDGE_FLOORS_SINGLE):
    height_ratio = Z_COORDS[floor_top] / BUILDING_HEIGHT
    bridge_elems = create_box_bridge(floor_bot, floor_top, f"1-Story Bridge {i+1}")
    bridge_elements.extend(bridge_elems)

# 2-story bridge at top (2-floor span)
floor_bot, floor_top = BRIDGE_FLOORS_DOUBLE
two_story_elems = create_box_bridge(floor_bot, floor_top, "2-Story Bridge")
bridge_elements.extend(two_story_elems)

print(f"\nTotal bridge elements: {len(bridge_elements)}")
print(f"Total bridge nodes: {len(bridge_nodes)}")

# Summary by element type
bridge_df_temp = pd.DataFrame(bridge_elements)
print("\nBridge element types:")
print(bridge_df_temp['element_type'].value_counts().to_string())

Building height: 153m = 1530mm in maket

Bridge Configuration:
  Single-story bridges (1-floor span): [(5, 6), (11, 12), (17, 18)]
  2-story bridge: (23, 25)
  Bridge X: 16 to 24m (width: 8m)
  Bridge Y: 16 to 24m (span: 8m)
  Height per floor: 6m

Creating bridge nodes...
  Created 16 bridge nodes

Creating bridges (all with box truss structure)...
  1-Story Bridge 1 (Floor 5→6, height=6m): 40 elements
  1-Story Bridge 2 (Floor 11→12, height=6m): 40 elements
  1-Story Bridge 3 (Floor 17→18, height=6m): 40 elements
  2-Story Bridge (Floor 23→25, height=12m): 40 elements

Total bridge elements: 160
Total bridge nodes: 16

Bridge element types:
element_type
bridge_beam          56
bridge_truss_side    32
bridge_column        24
bridge_brace_bot     16
bridge_brace_top     16
bridge_brace_face    16


---
## 6. Combine All Elements

In [140]:
# ============================================
# ADD BRIDGE NODES TO COORDS AND POSITION DF
# ============================================
# Extend coords array with bridge nodes
if len(bridge_nodes) > 0:
    bridge_nodes_df = pd.DataFrame(bridge_nodes)
    n_total_nodes = n_nodes + len(bridge_nodes)
    
    # Expand coords array
    new_coords = np.zeros((n_total_nodes, 3))
    new_coords[:n_nodes] = coords
    for i, bn in enumerate(bridge_nodes):
        new_coords[bn['node_id']] = [bn['x'], bn['y'], bn['z']]
    coords = new_coords
    
    # Add bridge nodes to position_df
    position_df = pd.concat([position_df, bridge_nodes_df], ignore_index=True)
    
    print(f"Added {len(bridge_nodes)} bridge nodes")
    print(f"Total nodes: {n_total_nodes}")
else:
    n_total_nodes = n_nodes

# ============================================
# COMBINE ALL ELEMENTS
# ============================================
all_elements = elements_t1 + elements_t2 + bridge_elements

# Calculate element lengths
for elem in all_elements:
    n1, n2 = elem['node_i'], elem['node_j']
    c1, c2 = coords[n1], coords[n2]
    length = np.sqrt(np.sum((c2 - c1) ** 2))
    elem['length'] = round(length, 4)

# Add connection type default for tower elements
for elem in all_elements:
    if 'connection' not in elem:
        elem['connection'] = 'rigid'

connectivity_df = pd.DataFrame(all_elements)

print("\n" + "=" * 70)
print("TWIN TOWERS CONNECTIVITY SUMMARY")
print("=" * 70)
print(f"\nTotal nodes: {n_total_nodes}")
print(f"  Tower 1: {len(nodes_t1)} nodes")
print(f"  Tower 2: {len(nodes_t2)} nodes")
print(f"  Bridge:  {len(bridge_nodes)} nodes")

print(f"\nTotal elements: {len(connectivity_df)}")
print(f"\nElements by type:")
print(connectivity_df['element_type'].value_counts().to_string())
print(f"\nElements by tower/bridge:")
print(connectivity_df['tower'].value_counts())

# Connection types summary
print(f"\nConnection types:")
print(connectivity_df['connection'].value_counts())

# Bridge elements detail
bridge_elem_df = connectivity_df[connectivity_df['tower'] == 'bridge']
print(f"\nBridge element details ({len(bridge_elem_df)} total):")
print(bridge_elem_df['element_type'].value_counts().to_string())

Added 16 bridge nodes
Total nodes: 796

TWIN TOWERS CONNECTIVITY SUMMARY

Total nodes: 796
  Tower 1: 390 nodes
  Tower 2: 390 nodes
  Bridge:  16 nodes

Total elements: 2768

Elements by type:
element_type
column               744
beam_x               624
beam_y               520
brace_xz             216
brace_floor          116
outrigger             96
core_wall             80
belt_truss            80
brace_yz              64
bridge_beam           56
brace_space           36
brace_corner          32
bridge_truss_side     32
bridge_column         24
bridge_brace_bot      16
bridge_brace_top      16
bridge_brace_face     16

Elements by tower/bridge:
tower
1         1304
2         1304
bridge     160
Name: count, dtype: int64

Connection types:
connection
rigid    2624
pin       144
Name: count, dtype: int64

Bridge element details (160 total):
element_type
bridge_beam          56
bridge_truss_side    32
bridge_column        24
bridge_brace_bot     16
bridge_brace_top     16
bridge_bra

---
## 7. 3D Visualization

In [143]:
# 3D Twin Towers Visualization
fig_3d = go.Figure()

# Element styles for visualization
element_styles = {
    # Tower elements
    'column':           {'color': 'blue',         'width': 3},
    'beam_x':           {'color': 'green',        'width': 2},
    'beam_y':           {'color': 'orange',       'width': 2},
    'brace_xz':         {'color': 'red',          'width': 2},
    'brace_yz':         {'color': 'purple',       'width': 2},
    'brace_floor':      {'color': 'lightgray',    'width': 1},
    'brace_space':      {'color': 'black',        'width': 4},
    'core_wall':        {'color': 'brown',        'width': 3},
    'brace_corner':     {'color': 'darkblue',     'width': 3},
    'brace_transition': {'color': 'gold',         'width': 3},
    # Outrigger system
    'outrigger':        {'color': 'darkorange',   'width': 4},
    'belt_truss':       {'color': 'darkgreen',    'width': 4},
    # Bridge elements
    'bridge_beam':      {'color': 'magenta',      'width': 5},
    'bridge_column':    {'color': 'darkmagenta',  'width': 4},
    'bridge_brace_top': {'color': 'hotpink',      'width': 3},
    'bridge_brace_bot': {'color': 'deeppink',     'width': 3},
    'bridge_truss_side':{'color': 'fuchsia',      'width': 3},
    'bridge_brace_face':{'color': 'mediumvioletred', 'width': 3},
    'bridge_brace':     {'color': 'magenta',      'width': 3},  # fallback
}

for etype, style in element_styles.items():
    elements = connectivity_df[connectivity_df['element_type'] == etype]
    if len(elements) == 0:
        continue
    
    x_lines, y_lines, z_lines = [], [], []
    for _, row in elements.iterrows():
        n1_coords = coords[int(row['node_i'])]
        n2_coords = coords[int(row['node_j'])]
        x_lines.extend([n1_coords[0], n2_coords[0], None])
        y_lines.extend([n1_coords[1], n2_coords[1], None])
        z_lines.extend([n1_coords[2], n2_coords[2], None])
    
    fig_3d.add_trace(go.Scatter3d(
        x=x_lines, y=y_lines, z=z_lines,
        mode='lines',
        name=f"{etype.replace('_', ' ').title()} ({len(elements)})",
        line=dict(color=style['color'], width=style['width']),
        hoverinfo='name'
    ))

# Add bridge nodes
zone_nodes = position_df[position_df['zone'] == 'bridge']
if len(zone_nodes) > 0:
    fig_3d.add_trace(go.Scatter3d(
        x=zone_nodes['x'], y=zone_nodes['y'], z=zone_nodes['z'],
        mode='markers',
        marker=dict(size=5, color='magenta'),
        name=f'Bridge Nodes ({len(zone_nodes)})',
        hoverinfo='text',
        hovertext=[f"N{int(r['node_id'])}: ({r['x']:.0f},{r['y']:.0f},{r['z']:.0f})" 
                   for _, r in zone_nodes.iterrows()]
    ))

# Add all tower nodes
tower_nodes = position_df[position_df['zone'] != 'bridge']
fig_3d.add_trace(go.Scatter3d(
    x=tower_nodes['x'], y=tower_nodes['y'], z=tower_nodes['z'],
    mode='markers',
    marker=dict(size=2, color='gray', opacity=0.5),
    name=f'Tower Nodes ({len(tower_nodes)})',
    hoverinfo='text',
    hovertext=[f"N{int(r['node_id'])}: ({r['x']:.0f},{r['y']:.0f},{r['z']:.0f})" 
               for _, r in tower_nodes.iterrows()]
))

# Label bridges - BRIDGE_FLOORS_SINGLE is now list of tuples
for floor_bot, floor_top in BRIDGE_FLOORS_SINGLE:
    z_mid = (Z_COORDS[floor_bot] + Z_COORDS[floor_top]) / 2
    fig_3d.add_trace(go.Scatter3d(
        x=[20], y=[20], z=[z_mid],
        mode='text',
        text=[f'Bridge F{floor_bot}-{floor_top}'],
        textposition='middle center',
        showlegend=False
    ))

# Label 2-story bridge
z_mid = (Z_COORDS[BRIDGE_FLOORS_DOUBLE[0]] + Z_COORDS[BRIDGE_FLOORS_DOUBLE[1]]) / 2
fig_3d.add_trace(go.Scatter3d(
    x=[20], y=[20], z=[z_mid],
    mode='text',
    text=['2-Story Bridge'],
    textposition='middle center',
    showlegend=False
))

fig_3d.update_layout(
    title='DASK 2026 Twin Towers with Box Truss Bridges',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data',
        camera=dict(eye=dict(x=1.5, y=1.5, z=0.8))
    ),
    width=1200, height=900,
    legend=dict(x=1.02, y=1, bgcolor='rgba(255,255,255,0.8)')
)
fig_3d.show()

---
## 7.1 Front & Back Face Elevation Views

- **Front face**: Y = 0 (Tower 1 front) and Y = 24 (Tower 2 front / bridge interface)
- **Back face**: Y = 16 (Tower 1 back / bridge interface) and Y = 40 (Tower 2 back)

In [57]:
# ============================================
# FRONT AND BACK FACE ELEVATION VIEWS
# ============================================
from plotly.subplots import make_subplots

# Define face Y coordinates
FRONT_FACES_Y = [Y_COORDS_T1[0], Y_COORDS_T2[0]]    # Y=0 (T1 front), Y=24 (T2 front)
BACK_FACES_Y = [Y_COORDS_T1[-1], Y_COORDS_T2[-1]]   # Y=16 (T1 back), Y=40 (T2 back)

def filter_elements_by_y(df, coords, y_values, tolerance=0.5):
    """Filter elements where both nodes are at given Y values (within tolerance)"""
    mask = []
    for _, row in df.iterrows():
        n1, n2 = int(row['node_i']), int(row['node_j'])
        y1, y2 = coords[n1][1], coords[n2][1]
        
        # Check if both nodes are at one of the Y values (front/back face)
        at_face_1 = any(abs(y1 - y) < tolerance for y in y_values)
        at_face_2 = any(abs(y2 - y) < tolerance for y in y_values)
        
        # Also include vertical elements at this Y (XZ plane)
        same_y = abs(y1 - y2) < tolerance
        mask.append((at_face_1 and at_face_2) or (same_y and at_face_1))
    
    return df[mask]

def create_elevation_view(df, coords, face_name, y_filter_values):
    """Create 2D elevation view (XZ plane)"""
    filtered_df = filter_elements_by_y(df, coords, y_filter_values, tolerance=1.0)
    
    fig = go.Figure()
    
    # Element styles
    element_styles = {
        'column': {'color': 'blue', 'width': 3},
        'beam_x': {'color': 'green', 'width': 2},
        'beam_y': {'color': 'orange', 'width': 2},
        'brace_xz': {'color': 'red', 'width': 2},
        'brace_yz': {'color': 'purple', 'width': 2},
        'core_wall': {'color': 'brown', 'width': 3},
        'brace_corner': {'color': 'darkblue', 'width': 3},
        'brace_transition': {'color': 'gold', 'width': 3},
        'outrigger': {'color': 'darkorange', 'width': 4},
        'belt_truss': {'color': 'darkgreen', 'width': 4},
        'bridge_beam': {'color': 'magenta', 'width': 5},
        'bridge_column': {'color': 'darkmagenta', 'width': 4},
        'bridge_brace_face': {'color': 'mediumvioletred', 'width': 3},
        'bridge_truss_side': {'color': 'fuchsia', 'width': 3},
        'bridge_brace_top': {'color': 'hotpink', 'width': 3},
        'bridge_brace_bot': {'color': 'deeppink', 'width': 3},
    }
    
    for etype in filtered_df['element_type'].unique():
        style = element_styles.get(etype, {'color': 'gray', 'width': 1})
        elems = filtered_df[filtered_df['element_type'] == etype]
        
        x_lines, z_lines = [], []
        for _, row in elems.iterrows():
            n1, n2 = int(row['node_i']), int(row['node_j'])
            x_lines.extend([coords[n1][0], coords[n2][0], None])
            z_lines.extend([coords[n1][2], coords[n2][2], None])
        
        if x_lines:
            fig.add_trace(go.Scatter(
                x=x_lines, y=z_lines,
                mode='lines',
                name=f"{etype.replace('_', ' ').title()} ({len(elems)})",
                line=dict(color=style['color'], width=style['width']),
                hoverinfo='name'
            ))
    
    # Filter nodes
    node_mask = [any(abs(coords[i][1] - y) < 1.0 for y in y_filter_values) for i in range(len(coords))]
    filtered_nodes = position_df[node_mask]
    
    fig.add_trace(go.Scatter(
        x=filtered_nodes['x'], y=filtered_nodes['z'],
        mode='markers',
        marker=dict(size=3, color='gray', opacity=0.5),
        name=f'Nodes ({len(filtered_nodes)})',
        hoverinfo='text',
        hovertext=[f"N{int(r['node_id'])}: ({r['x']:.0f},{r['y']:.0f},{r['z']:.0f})" 
                   for _, r in filtered_nodes.iterrows()]
    ))
    
    fig.update_layout(
        title=f'{face_name} Elevation View (XZ Plane)',
        xaxis_title='X (m)',
        yaxis_title='Z (m)',
        yaxis=dict(scaleanchor='x', scaleratio=1),
        width=1000, height=800,
        legend=dict(x=1.02, y=1)
    )
    
    return fig, filtered_df

# Create Front Face View (Y=0 for T1, Y=24 for T2)
print("=" * 70)
print("FRONT FACE ELEVATION (Y=0 and Y=24)")
print("=" * 70)
fig_front, front_elems = create_elevation_view(
    connectivity_df, coords, 
    "FRONT FACE", 
    [Y_COORDS_T1[0], Y_COORDS_T2[0]]  # Y=0, Y=24
)
print(f"Front face elements: {len(front_elems)}")
print(front_elems['element_type'].value_counts().to_string())
fig_front.show()

# Create Back Face View (Y=16 for T1, Y=40 for T2)
print("\n" + "=" * 70)
print("BACK FACE ELEVATION (Y=16 and Y=40)")
print("=" * 70)
fig_back, back_elems = create_elevation_view(
    connectivity_df, coords, 
    "BACK FACE", 
    [Y_COORDS_T1[-1], Y_COORDS_T2[-1]]  # Y=16, Y=40
)
print(f"Back face elements: {len(back_elems)}")
print(back_elems['element_type'].value_counts().to_string())
fig_back.show()

FRONT FACE ELEVATION (Y=0 and Y=24)
Front face elements: 612
element_type
column               248
beam_x               208
brace_xz             128
bridge_beam            8
bridge_column          8
bridge_brace_face      8
brace_space            4



BACK FACE ELEVATION (Y=16 and Y=40)
Back face elements: 612
element_type
column               248
beam_x               208
brace_xz             128
bridge_beam            8
bridge_column          8
bridge_brace_face      8
brace_space            4


In [144]:
# ============================================
# EXPORT ALL VISUALIZATIONS
# ============================================
VIZ_DIR = '../results/visualizations'
os.makedirs(VIZ_DIR, exist_ok=True)

# Export 3D model view
fig_3d.write_html(os.path.join(VIZ_DIR, 'twin_towers_3d.html'))

# Export as interactive HTML
fig_front.write_html(os.path.join(VIZ_DIR, 'front_elevation.html'))
fig_back.write_html(os.path.join(VIZ_DIR, 'back_elevation.html'))

# Export as static images (PNG)
try:
    fig_3d.write_image(os.path.join(VIZ_DIR, 'twin_towers_3d.png'), width=1200, height=900, scale=2)
    fig_front.write_image(os.path.join(VIZ_DIR, 'front_elevation.png'), width=1200, height=900, scale=2)
    fig_back.write_image(os.path.join(VIZ_DIR, 'back_elevation.png'), width=1200, height=900, scale=2)
    print("Exported PNG images")
except Exception as e:
    print(f"PNG export requires kaleido: {e}")
    print("Install with: pip install kaleido")

print(f"\nExported visualizations to {os.path.abspath(VIZ_DIR)}:")
print("  - twin_towers_3d.html")
print("  - front_elevation.html")
print("  - back_elevation.html")

PNG export requires kaleido: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Install with: pip install kaleido

Exported visualizations to /mnt/c/Users/lenovo/Desktop/DASK_NEW/results/visualizations:
  - twin_towers_3d.html
  - front_elevation.html
  - back_elevation.html


---
## 8. Export Data

In [142]:
# Export to ../data/ folder
DATA_DIR = '../data'
os.makedirs(DATA_DIR, exist_ok=True)

# Save with twin_ prefix to distinguish from single tower
position_df.to_csv(os.path.join(DATA_DIR, 'twin_position_matrix.csv'), index=False)
connectivity_df.to_csv(os.path.join(DATA_DIR, 'twin_connectivity_matrix.csv'), index=False)

# Adjacency matrix (use total nodes including bridge nodes)
adjacency_matrix = np.zeros((n_total_nodes, n_total_nodes), dtype=int)
for _, row in connectivity_df.iterrows():
    i, j = int(row['node_i']), int(row['node_j'])
    if i < n_total_nodes and j < n_total_nodes:
        adjacency_matrix[i, j] = 1
        adjacency_matrix[j, i] = 1

np.savetxt(os.path.join(DATA_DIR, 'twin_adjacency_matrix.csv'), adjacency_matrix, delimiter=',', fmt='%d')

# Save comprehensive data
np.savez(os.path.join(DATA_DIR, 'twin_building_data.npz'),
         adjacency_matrix=adjacency_matrix,
         coords=coords,
         podium_x=PODIUM_X_COORDS,
         tower_x=TOWER_X_COORDS,
         y_coords_t1=Y_COORDS_T1,
         y_coords_t2=Y_COORDS_T2,
         z_coords=Z_COORDS,
         tower_gap=TOWER_GAP,
         bridge_floors_single=np.array(BRIDGE_FLOORS_SINGLE),
         bridge_floors_double=np.array(BRIDGE_FLOORS_DOUBLE))

print(f"Exported to {os.path.abspath(DATA_DIR)}:")
print("  - twin_position_matrix.csv")
print("  - twin_connectivity_matrix.csv")
print("  - twin_adjacency_matrix.csv")
print("  - twin_building_data.npz")
print(f"\nTotal nodes: {n_total_nodes}")
print(f"Total elements: {len(connectivity_df)}")
print(f"Bridge elements: {len(connectivity_df[connectivity_df['tower'] == 'bridge'])}")

# Connection types
print(f"\nConnection summary:")
pin_conns = len(connectivity_df[connectivity_df['connection'] == 'pin'])
rigid_conns = len(connectivity_df[connectivity_df['connection'] == 'rigid'])
print(f"  Pin connections: {pin_conns}")
print(f"  Rigid connections: {rigid_conns}")

Exported to /mnt/c/Users/lenovo/Desktop/DASK_NEW/data:
  - twin_position_matrix.csv
  - twin_connectivity_matrix.csv
  - twin_adjacency_matrix.csv
  - twin_building_data.npz

Total nodes: 796
Total elements: 2768
Bridge elements: 160

Connection summary:
  Pin connections: 144
  Rigid connections: 2624


---
## 9. Summary Statistics

In [141]:
print("=" * 70)
print("TWIN TOWERS MODEL SUMMARY")
print("=" * 70)

print(f"\nGEOMETRY (Model Scale):")
print(f"  Tower 1: X=[0-40]m, Y=[0-16]m")
print(f"  Tower 2: X=[0-40]m, Y=[24-40]m")
print(f"  Gap: {TOWER_GAP}m = {TOWER_GAP*10:.0f}mm in maket")
print(f"  Height: {Z_COORDS[-1]:.0f}m = {Z_COORDS[-1]*10:.0f}mm in maket")

print(f"\nNODES: {n_nodes}")
print(f"  Tower 1: {len(nodes_t1)}")
print(f"  Tower 2: {len(nodes_t2)}")

print(f"\nELEMENTS: {len(connectivity_df)}")
for etype, count in connectivity_df['element_type'].value_counts().items():
    print(f"  {etype:15s}: {count:5d}")

print(f"\nBRIDGES:")
print(f"  Single-story: {len(BRIDGE_FLOORS_SINGLE)} bridges at floors {BRIDGE_FLOORS_SINGLE}")
print(f"  2-story: 1 bridge at floors {BRIDGE_FLOORS_DOUBLE}")
print(f"  Bridge elements: {len(bridge_elements)}")

# Total lengths
total_length = connectivity_df['length'].sum()
print(f"\nTOTAL ELEMENT LENGTH: {total_length:.2f} m (model scale)")
print(f"                      {total_length*10:.0f} mm (maket scale)")

TWIN TOWERS MODEL SUMMARY

GEOMETRY (Model Scale):
  Tower 1: X=[0-40]m, Y=[0-16]m
  Tower 2: X=[0-40]m, Y=[24-40]m
  Gap: 8.0m = 80mm in maket
  Height: 153m = 1530mm in maket

NODES: 780
  Tower 1: 390
  Tower 2: 390

ELEMENTS: 2768
  column         :   744
  beam_x         :   624
  beam_y         :   520
  brace_xz       :   216
  brace_floor    :   116
  outrigger      :    96
  core_wall      :    80
  belt_truss     :    80
  brace_yz       :    64
  bridge_beam    :    56
  brace_space    :    36
  brace_corner   :    32
  bridge_truss_side:    32
  bridge_column  :    24
  bridge_brace_bot:    16
  bridge_brace_top:    16
  bridge_brace_face:    16

BRIDGES:
  Single-story: 3 bridges at floors [(5, 6), (11, 12), (17, 18)]
  2-story: 1 bridge at floors (23, 25)
  Bridge elements: 160

TOTAL ELEMENT LENGTH: 23945.90 m (model scale)
                      239459 mm (maket scale)


---
## 10. Weight Analysis & 1.4 kg Limit Verification

DASK 2026 Competition Requirement:
- **Maximum model weight: 1.4 kg** (excluding roof mass and floor plates)
- Material: Balsa wood 6mm × 6mm bars
- Density varies: 100-200 kg/m³ typical range

In [ ]:
# ============================================
# WEIGHT ANALYSIS - 1.4 kg LIMIT VERIFICATION
# ============================================
from tabulate import tabulate

# Material properties
BALSA_SECTION = 6.0  # mm
BALSA_AREA = BALSA_SECTION ** 2  # 36 mm²
WEIGHT_LIMIT_KG = 1.40  # DASK 2026 limit

# Total element length in maket scale (mm)
total_length_mm = connectivity_df['length'].sum() * 10  # Convert model (m) to maket (mm)

print("=" * 70)
print("WEIGHT ANALYSIS - DASK 2026")
print("=" * 70)
print(f"\nTotal element length: {total_length_mm:,.0f} mm")
print(f"Section: {BALSA_SECTION:.0f}mm × {BALSA_SECTION:.0f}mm (A = {BALSA_AREA:.0f} mm²)")
print(f"Weight limit: {WEIGHT_LIMIT_KG} kg")

# Weight for different balsa densities
densities = [100, 120, 140, 160, 180, 200]
weight_data = []

for rho in densities:
    # Weight = Length × Area × Density
    # L (mm) × A (mm²) × rho (kg/m³) / 1e9 = kg
    weight_kg = total_length_mm * BALSA_AREA * rho / 1e9
    status = "✓ OK" if weight_kg <= WEIGHT_LIMIT_KG else "✗ OVER"
    margin_g = (WEIGHT_LIMIT_KG - weight_kg) * 1000
    
    weight_data.append({
        'Density (kg/m³)': rho,
        'Weight (g)': round(weight_kg * 1000, 1),
        'Weight (kg)': round(weight_kg, 3),
        'Margin (g)': round(margin_g, 1),
        'Status': status
    })

weight_df = pd.DataFrame(weight_data)
print("\n" + tabulate(weight_df, headers='keys', tablefmt='grid', showindex=False))

# Maximum allowable density
max_density = WEIGHT_LIMIT_KG * 1e9 / (total_length_mm * BALSA_AREA)
print(f"\n★ Maximum allowable balsa density: {max_density:.0f} kg/m³")

# Typical balsa (160 kg/m³) analysis
typical_weight = total_length_mm * BALSA_AREA * 160 / 1e9
print(f"\nWith typical balsa (ρ = 160 kg/m³):")
print(f"  Total weight: {typical_weight:.3f} kg = {typical_weight*1000:.0f} g")
if typical_weight <= WEIGHT_LIMIT_KG:
    print(f"  Status: ✓ UNDER LIMIT (margin: {(WEIGHT_LIMIT_KG - typical_weight)*1000:.0f} g)")
else:
    print(f"  Status: ✗ OVER LIMIT (excess: {(typical_weight - WEIGHT_LIMIT_KG)*1000:.0f} g)")

# Weight breakdown by component
print("\n" + "-" * 50)
print("WEIGHT BREAKDOWN BY COMPONENT (ρ = 160 kg/m³)")
print("-" * 50)

component_weights = []
for etype in connectivity_df['element_type'].unique():
    elems = connectivity_df[connectivity_df['element_type'] == etype]
    length_mm = elems['length'].sum() * 10  # m to mm
    weight_g = length_mm * BALSA_AREA * 160 / 1e6  # to grams
    pct = weight_g / (typical_weight * 1000) * 100
    component_weights.append({
        'Element Type': etype,
        'Count': len(elems),
        'Length (mm)': round(length_mm, 0),
        'Weight (g)': round(weight_g, 1),
        '% of Total': round(pct, 1)
    })

component_df = pd.DataFrame(component_weights).sort_values('Weight (g)', ascending=False)
print(tabulate(component_df, headers='keys', tablefmt='grid', showindex=False))

# Tower vs Bridge weight
tower_elems = connectivity_df[connectivity_df['tower'] != 'bridge']
bridge_elems = connectivity_df[connectivity_df['tower'] == 'bridge']

tower_weight_g = tower_elems['length'].sum() * 10 * BALSA_AREA * 160 / 1e6
bridge_weight_g = bridge_elems['length'].sum() * 10 * BALSA_AREA * 160 / 1e6

print(f"\nTOWERS vs BRIDGES (ρ = 160 kg/m³):")
print(f"  Both towers: {tower_weight_g:.1f} g ({tower_weight_g/10:.2f} kg)")
print(f"  All bridges: {bridge_weight_g:.1f} g ({bridge_weight_g/10:.2f} kg)")
print(f"  Total: {tower_weight_g + bridge_weight_g:.1f} g")

---
## 11. Period vs Sae(T) Design Spectrum

TBDY 2018 (Turkish Building Seismic Code) Horizontal Elastic Design Spectrum for DASK 2026:
- **Location**: ITU Ayazağa Campus, Istanbul
- **SDS** = 2.046 g (Short period design spectral acceleration)
- **SD1** = 0.619 g (1-second design spectral acceleration)
- **PGA** = 0.82 g (Peak Ground Acceleration)

The building's fundamental period determines its position on the spectrum and thus the seismic demand.

In [ ]:
# ============================================
# PERIOD vs Sae(T) DESIGN SPECTRUM
# TBDY 2018 Horizontal Elastic Design Spectrum
# ============================================

# DASK 2026 Spectrum Parameters (from şartname)
SDS = 2.046    # Short period design spectral acceleration (g)
SD1 = 0.619    # 1-second design spectral acceleration (g)
TA = 0.2 * SD1 / SDS   # Corner period TA
TB = SD1 / SDS         # Corner period TB
TL = 6.0               # Long period limit
PGA = 0.82             # Peak Ground Acceleration (g)

print("=" * 70)
print("TBDY 2018 DESIGN SPECTRUM - DASK 2026")
print("=" * 70)
print(f"\nSpectrum Parameters:")
print(f"  SDS = {SDS:.3f} g  (Short period spectral acceleration)")
print(f"  SD1 = {SD1:.3f} g  (1-second spectral acceleration)")
print(f"  TA  = {TA:.4f} s  (Ascending-Plateau corner)")
print(f"  TB  = {TB:.4f} s  (Plateau-Descending corner)")
print(f"  TL  = {TL:.1f} s  (Long period limit)")
print(f"  PGA = {PGA:.2f} g  (Peak Ground Acceleration)")

def Sae(T):
    """TBDY 2018 Horizontal Elastic Design Spectrum - returns Sae(T) in g"""
    if T < TA:
        return SDS * (0.4 + 0.6 * T / TA)
    elif T <= TB:
        return SDS
    elif T <= TL:
        return SD1 / T
    else:
        return SD1 * TL / (T * T)

# Generate spectrum curve
T_range = np.concatenate([
    np.linspace(0.001, TA, 20),
    np.linspace(TA, TB, 30),
    np.linspace(TB, 2.0, 100),
    np.linspace(2.0, 4.0, 50)
])
T_range = np.unique(np.sort(T_range))
Sae_values = np.array([Sae(T) for T in T_range])

# Estimated building period (from modal analysis)
# For a balsa model ~1.5m tall, typical T1 ~ 0.1-0.2 seconds
T_BUILDING = 0.1181  # seconds (estimated from previous analysis)
Sae_building = Sae(T_BUILDING)

# Determine spectrum region
if T_BUILDING < TA:
    region = "ASCENDING BRANCH (T < TA)"
    region_desc = "Period increases → Sae increases"
elif T_BUILDING <= TB:
    region = "PLATEAU (TA ≤ T ≤ TB)"
    region_desc = "Maximum spectral acceleration zone (Sae = SDS)"
else:
    region = "DESCENDING BRANCH (T > TB)"
    region_desc = "Period increases → Sae decreases"

print(f"\n" + "-" * 50)
print("BUILDING DYNAMIC PROPERTIES")
print("-" * 50)
print(f"  Fundamental Period T₁ = {T_BUILDING:.4f} s")
print(f"  Spectral Acceleration Sae(T₁) = {Sae_building:.3f} g")
print(f"  Spectrum Region: {region}")
print(f"  Implication: {region_desc}")

# Create Plotly figure
fig_spectrum = go.Figure()

# Design spectrum curve
fig_spectrum.add_trace(go.Scatter(
    x=T_range, y=Sae_values,
    mode='lines',
    name='TBDY 2018 Design Spectrum',
    line=dict(color='blue', width=3)
))

# Fill regions with different colors
# Ascending region
mask_asc = T_range <= TA
fig_spectrum.add_trace(go.Scatter(
    x=np.concatenate([T_range[mask_asc], T_range[mask_asc][::-1]]),
    y=np.concatenate([Sae_values[mask_asc], np.zeros(sum(mask_asc))]),
    fill='toself',
    fillcolor='rgba(0,255,0,0.2)',
    line=dict(width=0),
    name='Ascending (T < TA)',
    showlegend=True
))

# Plateau region
mask_plat = (T_range >= TA) & (T_range <= TB)
fig_spectrum.add_trace(go.Scatter(
    x=np.concatenate([T_range[mask_plat], T_range[mask_plat][::-1]]),
    y=np.concatenate([Sae_values[mask_plat], np.zeros(sum(mask_plat))]),
    fill='toself',
    fillcolor='rgba(255,165,0,0.2)',
    line=dict(width=0),
    name='Plateau (TA ≤ T ≤ TB)',
    showlegend=True
))

# Descending region
mask_desc = T_range >= TB
fig_spectrum.add_trace(go.Scatter(
    x=np.concatenate([T_range[mask_desc], T_range[mask_desc][::-1]]),
    y=np.concatenate([Sae_values[mask_desc], np.zeros(sum(mask_desc))]),
    fill='toself',
    fillcolor='rgba(0,0,255,0.2)',
    line=dict(width=0),
    name='Descending (T > TB)',
    showlegend=True
))

# Corner period lines
fig_spectrum.add_vline(x=TA, line_dash="dash", line_color="gray", 
                       annotation_text=f"TA={TA:.3f}s", annotation_position="top")
fig_spectrum.add_vline(x=TB, line_dash="dashdot", line_color="gray",
                       annotation_text=f"TB={TB:.3f}s", annotation_position="top")

# Building period marker
fig_spectrum.add_trace(go.Scatter(
    x=[T_BUILDING], y=[Sae_building],
    mode='markers',
    name=f'Building T₁ = {T_BUILDING:.4f}s',
    marker=dict(size=20, color='red', symbol='circle',
                line=dict(width=3, color='black'))
))

# Annotation for building
fig_spectrum.add_annotation(
    x=T_BUILDING, y=Sae_building,
    text=f"T₁ = {T_BUILDING:.4f}s<br>Sae = {Sae_building:.3f}g",
    showarrow=True,
    arrowhead=2,
    arrowcolor='red',
    ax=80, ay=-40,
    bgcolor='yellow',
    bordercolor='black',
    font=dict(size=12, color='black')
)

# SDS and PGA lines
fig_spectrum.add_hline(y=SDS, line_dash="dot", line_color="orange",
                       annotation_text=f"SDS={SDS:.3f}g", annotation_position="right")
fig_spectrum.add_hline(y=PGA, line_dash="dot", line_color="red",
                       annotation_text=f"PGA={PGA:.2f}g", annotation_position="right")

fig_spectrum.update_layout(
    title=dict(
        text='DASK 2026 - Period vs Spectral Acceleration (Sae)<br><sup>TBDY 2018 Horizontal Elastic Design Spectrum</sup>',
        font=dict(size=16)
    ),
    xaxis_title='Period T (s)',
    yaxis_title='Spectral Acceleration Sae(T) (g)',
    xaxis=dict(range=[0, 2], dtick=0.2),
    yaxis=dict(range=[0, max(Sae_values) * 1.15]),
    width=1000, height=600,
    legend=dict(x=0.65, y=0.95),
    template='plotly_white'
)

fig_spectrum.show()

# Save spectrum plot
fig_spectrum.write_html(os.path.join(VIZ_DIR, 'period_vs_sae_spectrum.html'))
print(f"\nSpectrum plot saved to: {VIZ_DIR}/period_vs_sae_spectrum.html")

# Analysis summary
print(f"""
╔══════════════════════════════════════════════════════════════════════╗
║                    SPECTRUM POSITION ANALYSIS                        ║
╠══════════════════════════════════════════════════════════════════════╣
║  Building Period:  T₁ = {T_BUILDING:.4f} s                                   ║
║  Spectrum Region:  {region:40s}   ║
║  Sae(T₁):          {Sae_building:.3f} g                                         ║
║  Sae/SDS ratio:    {Sae_building/SDS*100:.1f}% of plateau                               ║
╠══════════════════════════════════════════════════════════════════════╣
║  STRUCTURAL IMPLICATIONS:                                            ║
║  • Building is VERY STIFF (short fundamental period)                 ║
║  • Located in ascending branch where Sae increases with period       ║
║  • Adding stiffness (reducing T) → DECREASES spectral demand         ║
║  • Current demand: {Sae_building:.3f}g (less than plateau max {SDS:.3f}g)       ║
╚══════════════════════════════════════════════════════════════════════╝
""")